In [1]:

import h5py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
import deepdish
import scipy.interpolate as sci
from astropy.io import fits
from scipy import ndimage
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [2]:
#Load data from files

file = h5py.File('mgn_ng_1024.hdf5','r')
data = file['cube_display']
coords_file = h5py.File('t_h_plot_coords_20221108152301','r')
coords_from_file = coords_file['x_coords_all']

cube = data[550:675,0:550,145:270]
n = cube.shape[2]
#sobel = ndimage(cube,axis=2)

In [3]:
#Set up coordinates

coords_dict = {}
x_coords_all = coords_from_file
dict_group_load = coords_file['coords_dict']
dict_group_keys = dict_group_load.keys()
for i in dict_group_keys:
    coords_dict[i] = dict_group_load[i][:]
base_x = 485.0
base_y = 3.0
prev_coords = np.array([base_x,base_y])
i1 = None
i2 = None

In [4]:
fig, ax = plt.subplots()
ax.set_aspect(0.1)

x_vals_all = [*set(x_coords_all)]
x_range = np.arange(min(x_vals_all),max(x_vals_all)+1,1)
keys = list(coords_dict.keys())
#print(keys)
ints = []
n_y = len(x_range)
sheet_cube = np.zeros((n_y,n))
dist_cube = np.zeros((n_y,n))
coordinates = np.zeros((n_y,n))
interp_sheet = np.zeros((n_y,n))
for i in range(len(keys)):
    key = keys[i][:keys[i].find('_')]
    key = int(key)
    ints.append(key)
ints_no_dups = [*set(ints)]
ints_no_dups = sorted(ints_no_dups)
#print(ints_no_dups)
for j in ints_no_dups:
    x_vals = coords_dict[str(j)+'_x']
    y_vals = coords_dict[str(j)+'_y']
    z = np.polyfit(x_vals, y_vals, 3)
    f = np.poly1d(z)
    x_start = min(x_vals_all)
    x_end = max(x_vals_all)
    y_new = f(x_range)
    if j == min(ints_no_dups):
        y_coords_0 = y_new
    else:
        pass
    for k in range(len(x_range)):
        #print('k='+str(k))
        x0 = int(x_range[k])
        y0 = int(y_new[k])
        #prev_coords = np.array([base_x,base_y])
        if k == max(x_range):
            pix_val = ((cube[y0-1,x0,j]/2)+cube[y0,x0,j])/2
            b = np.array([x0,y0])
            a = prev_coords
            distance = np.linalg.norm(a-b)
            dist_cube[k,j] = distance
            prev_coords = b
        elif k == min(x_range):
            pix_val = ((cube[y0+1,x0,j]/2)+cube[y0,x0,j])/2
            prev_dist = np.array([base_x,base_y])
            a = prev_dist
            b = np.array([x0,y0])
            distance = np.linalg.norm(a-b)
            dist_cube[k,j] = distance
            prev_coords = b
        else:
            pix_val = ((cube[y0-1,x0,j]/2)+(cube[y0+1,x0,j]/2)+cube[y0,x0,j])/3
            a = prev_coords
            b = np.array([x0,y0])
            distance = np.linalg.norm(a-b)
            dist_cube[k,j] = distance
            prev_coords = b
        sheet_cube[k,j] = pix_val
        coordinates[k,j] = y0
for x in range(n):
    if x < min(ints_no_dups):
        for y in range(len(x_range)):
            x0 = int(x_range[y])
            y0 = int(coordinates[y,x])
            pix_val = cube[y0,x0,x]
            sheet_cube[y,x] = pix_val 
            coordinates[y,x] = y0
            if y == 0:
                prev_coords = np.array([base_x,base_y])
            else:
                pass 
            a = prev_coords 
            b = np.array((x0,y0))
            distance = np.linalg.norm(a-b)
            dist_cube[y,x] = distance 
            prev_coords = b
    elif x in ints_no_dups:
        if x == max(ints_no_dups):
            pass 
        else:
            i1 = int(x)
            i2_index = int(ints_no_dups.index(x)+1)
            i2 = ints_no_dups[i2_index]
    else:
        for y in range(len(x_range)):
            x0 = int(x_range[y])
            i_diff = i2 - i1
            y1 = coordinates[y,i1]
            y2 = coordinates[y,i2]
            #print('y='+str(y))
            #print('i1='+str(self.i1))
            #print('i2='+str(self.i2))
            #print('y1='+str(y1))
            #print('y2='+str(y2))
            y_diff = y2 - y1
            ints_0 = y_diff/i_diff
            y_prev = coordinates[y,x-1]
            #print('y_prev='+str(y_prev))
            if y_diff == 0:
                y0 = int(y_prev)
            else: 
                y0 = int(round(y_prev + ints_0))
            pix_val = ((cube[y0-1,x0,x]/2)+(cube[y0+1,x0,x]/2)+cube[y0,x0,x])/3
            sheet_cube[y,x] = pix_val 
            coordinates[y,x] = y0
            if y == 0:
                prev_coords = np.array([base_x,base_y])
            else:
                pass 
            a = prev_coords
            b = np.array((x0, y0))
            distance = np.linalg.norm(a-b)
            dist_cube[y,x] = distance 
            prev_coords = b
ax.imshow(np.flip(sheet_cube,axis=0), cmap = 'viridis',vmax = 0.5)
ax.set_aspect('auto')
ax.invert_yaxis()
fig.canvas.draw()
plt.show()
#print(self.sheet_cube.shape)
#print(self.sheet_cube)
#print(self.dist_cube.shape)
#print(self.dist_cube)


In [5]:
distances = np.zeros((dist_cube.shape[0],dist_cube.shape[1]))
dist_cube2 = np.flip(dist_cube,axis=0)
for i in range(dist_cube.shape[1]):
    for j in range(dist_cube.shape[0]):
        total = sum(dist_cube2[:j+1,i])
        distances[j,i] = total
distances = np.flip(distances,axis=0)

In [6]:
print(distances[100,80:100])

[433.5218919  365.13708499 364.72287143 409.63553218 363.48023074
 363.48023074 393.1458622  363.48023074 363.8944443  405.50654198
 363.8944443  364.30865787 373.78298812 365.13708499 365.13708499
 388.62237656 364.72287143 364.72287143 364.72287143 364.30865787]


In [7]:
times = np.arange(125)
times = np.repeat(times,432)

new_times = np.repeat(np.arange(125),450)

In [8]:
h = distances[:,45]

In [9]:
print(sheet_cube.shape)

(432, 125)


In [10]:
y = np.concatenate([distances[:, i] for i in range(125)])

In [11]:
values = np.concatenate([sheet_cube[:,i] for i in range(125)])

In [12]:
new_heights = np.concatenate([np.arange(450,0,-1) for i in range(125)])

In [13]:
test_grid = sci.griddata((times,y),values,(new_times,new_heights),method='cubic')

In [14]:
print(times.shape)
print(values.shape)
print(new_heights.shape)
print(y.shape)
print(new_times.shape)

(54000,)
(54000,)
(56250,)
(54000,)
(56250,)


In [15]:
test_grid.shape

(56250,)

In [16]:
test_grid2 = test_grid.reshape((125,450))

In [17]:
fig2, ax2 = plt.subplots()
plt.imshow(np.flip(np.rot90(test_grid2,k=3),axis=1))
ax2.set_aspect('auto')
plt.show()

In [18]:
fig3, ax3 = plt.subplots(nrows=2, sharex=True,sharey=True, figsize=(12,16))

ax3[0].imshow(sheet_cube**2,vmax = 0.2**2, vmin=0, cmap = 'plasma')
ax3[0].set_aspect('auto')
ax3[0].set_title('Original')

ax3[1].imshow(np.flip(np.rot90(test_grid2,k=3),axis=1)**2,vmax=0.2**2, vmin=0, cmap = 'plasma')
ax3[1].set_aspect('auto')
ax3[1].set_title('Griddata')

plt.show()

In [55]:
#pwd

In [40]:
#hdu = fits.PrimaryHDU(sheet_cube)

In [41]:
#hdu1 = fits.HDUList([hdu])

In [42]:
#AIA_flows_array = sheet_cube

In [43]:
#AIA_flows_array.shape

In [44]:
#hdu = fits.PrimaryHDU(AIA_flows_array)

In [45]:
#hdu1 = fits.HDUList([hdu])

In [46]:
#hdu1.writeto('AIA_flows.fits')

In [78]:
#Designate first structure

fig4, ax4 = plt.subplots(figsize=(12,10))

points_x = []
points_y = []

def onclick(event):
    ix, iy = event.xdata, event.ydata
    points_x.append(ix)
    points_y.append(iy)
    #print(ix, iy)

cid = fig4.canvas.mpl_connect('button_press_event', onclick) 

im = ax4.imshow(np.flip(np.rot90(test_grid2,k=3),axis=1)**2,vmax=0.2**2, vmin=0, cmap = 'plasma')
ax4.set_aspect('auto')
plt.show()

In [79]:
z = np.polyfit(points_x, points_y, 3)
x_range = np.arange(min(points_x), max(points_x)+1, 0.01)
f0 = np.poly1d(z)
y0 = f0(x_range)
x0 = x_range

In [80]:
fig5, ax5 = plt.subplots(figsize=(12,10))
im = ax5.imshow(np.flip(np.rot90(test_grid2,k=3),axis=1)**2,vmax=0.2**2, vmin=0, cmap = 'plasma')
plt.plot(x0,y0)
ax5.set_aspect('auto')

In [84]:
#Designate second structure

fig6, ax6 = plt.subplots(figsize=(12,10))

points_x = []
points_y = []

def onclick(event):
    ix, iy = event.xdata, event.ydata
    points_x.append(ix)
    points_y.append(iy)
    #print(ix, iy)

cid = fig6.canvas.mpl_connect('button_press_event', onclick) 

im = ax6.imshow(np.flip(np.rot90(test_grid2,k=3),axis=1)**2,vmax=0.2**2, vmin=0, cmap = 'plasma')
ax6.set_aspect('auto')
plt.show()

In [85]:
z = np.polyfit(points_x, points_y, 3)
x_range = np.arange(min(points_x), max(points_x)+1, 0.01)
f1 = np.poly1d(z)
y1 = f1(x_range)
x1 = x_range
fig7, ax7 = plt.subplots(figsize=(12,10))
im = ax7.imshow(np.flip(np.rot90(test_grid2,k=3),axis=1)**2,vmax=0.2**2, vmin=0, cmap = 'plasma')
plt.plot(x1,y1)
ax7.set_aspect('auto')